In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from place_groups import place_groups_middle_of_roads, place_groups_junctions

from sklearn.linear_model import LinearRegression
from datetime import timedelta
from keras.callbacks import ModelCheckpoint
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [41]:
DEFAULT_FLOW_FOR_MISSING_VALUES_AT_BEGINNING_OF_TIME_SERIES = 0
ONE_DAY_LAGS = 288
DATA_PATH = '/mnt/sdc1/inon/norrecampus/data/by_place_5min'

THRESHOLD_HIGH_OUTLIER = 110
LOSS = 'mse'
MINI_BATCH_SIZE = 512
NUM_EPOCHS = 100
VALIDATION_SPLIT = 0.2
NUM_LAGS = 12
LSTM_STATE_SIZE = NUM_LAGS
SPLIT_DATE = '2015-06-01'

In [4]:
def split_timestamp(ser):
    return pd\
        .DataFrame(ser)\
        .assign(day_of_week=lambda df: df.index.dayofweek, time_of_day=lambda df: df.index.time)\
        .reset_index()

In [5]:
def compute_error_statistics(errors_df, place_id):
    abs_errors = errors_df.error.abs()
    abs_errors_normalized = abs_errors / errors_df.flow_decile_true
    return pd.DataFrame({
        'corr': np.corrcoef(errors_df.flow_decile_predicted, errors_df.flow_decile_true)[0, 1],
        'mae': np.mean(abs_errors),
        'mape': np.mean(abs_errors_normalized),
        'mse': np.mean(abs_errors ** 2),
        'msne': np.mean(abs_errors_normalized ** 2),
        'rae': np.sum(abs_errors) / np.sum(np.abs(errors_df.flow_decile_true - np.mean(errors_df.flow_decile_true))),
        'rmse': np.sqrt(np.mean(abs_errors ** 2)),
        'rmsne': np.sqrt(np.mean(abs_errors_normalized ** 2)),
        'r2': max(0, 1 - np.sum(abs_errors ** 2) / np.sum((errors_df.flow_decile_true - np.mean(errors_df.flow_decile_true)) ** 2))
    }, index=[place_id])

In [6]:
def binary_encode(flow_decile):
    return ','.join('0' * (9 - int(flow_decile)) + '1' * (int(flow_decile) + 1))[::-1]

assert binary_encode(0) == '1,0,0,0,0,0,0,0,0,0'
assert binary_encode(9) == '1,1,1,1,1,1,1,1,1,1'
assert binary_encode(3) == '1,1,1,1,0,0,0,0,0,0'

In [7]:
def binary_decode(nn_result, threshold):
    for i in range(len(nn_result)):
        if nn_result[i] < threshold:
            return i - 1
    return len(nn_result) - 1

for i in range(10):
    assert binary_decode([1] * (i + 1) + [0] * (9 - i), 1) == i
assert binary_decode([0.6,0.7,0.55,0.51,0.2,1,0,0,0,1], 0.5) == 3

In [37]:
class LearnerForFlowDecileOfOneSegmentWithoutDetrending(object):
    def __init__(self,
                 threshold_ordinal_regression,
                 loss,
                 place_id, 
                 mini_batch_size,
                 num_epochs,
                 validation_split,
                 num_lags,
                 lstm_state_size,
                 split_date):
        self.df = None
        self.model = None
        
        self.threshold_ordinal_regression = threshold_ordinal_regression
        self.loss = loss
        self.place_id = place_id
        self.mini_batch_size = mini_batch_size
        self.num_epochs = num_epochs
        self.validation_split = validation_split
        self.num_lags = num_lags
        self.lstm_state_size = lstm_state_size
        self.split_date = split_date
        self.checkpoint_basename = 'ordinal_regression_flow_%s.best.hdf5' % self.place_id      
    
    def baseline_lr(self):
        X_train, Y_train, X_test, Y_test = learner._split_to_train_and_test()
        trained_lr = LinearRegression(fit_intercept=False).fit(X_train.values, Y_train.values)
        lr_predictions = trained_lr.predict(X_test).round(0)
        errors_df = self._individual_errors_without_interpolated_values(lr_predictions)
        return compute_error_statistics(errors_df, self.place_id)
                    
    def _imputation_for_missing_values(self, ser):
        missing_timestamps = pd\
            .date_range(min(ser.index.date), max(ser.index.date) + timedelta(days=1), freq='5T')\
            .difference(ser.index)
        df_with_nans_where_missing = pd.DataFrame(ser)\
            .join(pd.DataFrame(index=missing_timestamps), how='outer')\
            .assign(original_value=lambda df: df.iloc[:, 0])
        df_interpolated = df_with_nans_where_missing\
            .iloc[:, 0]\
            .interpolate()\
            .fillna(DEFAULT_FLOW_FOR_MISSING_VALUES_AT_BEGINNING_OF_TIME_SERIES)\
            .round()\
            .astype(np.int64)\
            .to_frame()\
            .assign(original_value=df_with_nans_where_missing.original_value)\
            .assign(is_interpolated=lambda df: df.iloc[:, 0] != df.iloc[:, 1])
        assert all(df_interpolated.flow_decile.isin(range(10)))
        return df_interpolated
        
    def _get_flows(self):
        if self.df is None:
            self.df = pd.read_csv(
                os.path.join(DATA_PATH, self.place_id + '.csv'),
                parse_dates=['start_interval_s', 'end_interval_s']\
            )[lambda df: df.start_interval_s >= '2015-01-01']\
            .rename(columns={'start_interval_s': 't', 'flow_bucket': 'flow_decile'})\
            .set_index('t')\
            .flow_decile
        return self.df

    def _split_to_train_and_test(self):
        flows_interpolated = self._imputation_for_missing_values(self._get_flows()).flow_decile
        lags = pd.concat([flows_interpolated.shift(x) for x in range(self.num_lags + 1)], axis=1)[self.num_lags:]
        train = lags[lags.index < self.split_date]
        X_train = train.iloc[:, 1:]
        Y_train = train.iloc[:, 0]
        test = lags[lags.index >= self.split_date]
        X_test = test.iloc[:, 1:]
        Y_test = test.iloc[:, 0]
        return X_train, Y_train, X_test, Y_test
    
    def _get_train_and_test_inputs(self):
        X_train, Y_train, X_test, Y_test = self._split_to_train_and_test()
        
        def bin_enc_ser(ser):
            return np.array(
                ser.apply(binary_encode).str.split(',', expand=True).astype(int))\
                [:, :, np.newaxis]

        def bin_enc(inp):
            return np.concatenate([bin_enc_ser(inp.iloc[:, i]) for i in range(inp.shape[1])], axis=2)\
                .swapaxes(1, 2)

        return \
            bin_enc(X_train), bin_enc_ser(Y_train).squeeze(axis=2), \
            bin_enc(X_test), bin_enc_ser(Y_test).squeeze(axis=2) \

    def _build_model(self):
        # TODO: better weights initialization
        self.model = Sequential()
        self.model.add(LSTM(self.lstm_state_size, input_shape=(self.num_lags, 10), return_sequences=False))
        # model.add(Dropout(0.2))
        #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
        self.model.add(Dense(units=10, activation="sigmoid"))
        self.model.compile(loss=self.loss, optimizer="rmsprop")  # TODO: try adam optimizer too, although rmsprop is the default go-to for RNN
    
    def load_best_model_from_disk(self):
        self._build_model()
        self.model.load_weights(self.checkpoint_basename)
    
    def create_and_train_model(self):
        self._build_model()
        X_train, Y_train, _, _ = self._get_train_and_test_inputs()
        self.model.fit(
            X_train,
            Y_train,
            batch_size=self.mini_batch_size,
            epochs=self.num_epochs,
            validation_split=self.validation_split,
            # checkpoint best model
            callbacks=[ModelCheckpoint(
                self.checkpoint_basename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')],
            verbose=2)
        self.model.load_weights(self.checkpoint_basename)
    
    def predict(self):
        _, _, X_test, _ = self._get_train_and_test_inputs()
        predictions_encoded = self.model.predict(X_test)
        predictions_decoded = np.apply_along_axis(
            lambda prediction: binary_decode(prediction, 0.5), 1, predictions_encoded)
        errors_df = self._individual_errors_without_interpolated_values(predictions_decoded)
        return compute_error_statistics(errors_df, self.place_id)
        
    def _individual_errors_without_interpolated_values(self, predictions_decoded):
        _, _, _, Y_test = self._split_to_train_and_test()
        ser_predictions_decoded = pd.Series(predictions_decoded, index=Y_test.index, name='flow_decile')
        interpolated_timestamps = self\
            ._imputation_for_missing_values(self._get_flows())\
            [lambda df: df.is_interpolated]\
            .index
        return Y_test.to_frame()\
            .join(ser_predictions_decoded.to_frame(), lsuffix='_true', rsuffix='_predicted')\
            .loc[lambda df: df.index.difference(interpolated_timestamps)]\
            .assign(error=lambda df: df.flow_decile_true - df.flow_decile_predicted)

In [39]:
def one_place(place_id):
    learner = LearnerForFlowDecileOfOneSegmentWithoutDetrending(
        threshold_ordinal_regression = 0.5,
        loss=LOSS,
        place_id=place_id, 
        mini_batch_size=MINI_BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        validation_split=VALIDATION_SPLIT,
        num_lags=NUM_LAGS,
        lstm_state_size=LSTM_STATE_SIZE,
        split_date=SPLIT_DATE)
    learner.create_and_train_model()
    return learner.predict()

def multiple_places(place_group, place_group_name):
    places = [lst[len(lst) // 2] for lst in place_group]
    return pd.concat(map(one_place, places))\
        .to_csv('%s_with_ordinal_regression.csv' % place_group_name)

In [10]:
multiple_places(place_groups_middle_of_roads, 'place_groups_middle_of_roads')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.13466, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 3s - loss: 0.1923 - val_loss: 0.1347
Epoch 2/100
Epoch 00002: val_loss improved from 0.13466 to 0.07897, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.1058 - val_loss: 0.0790
Epoch 3/100
Epoch 00003: val_loss improved from 0.07897 to 0.05513, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.0686 - val_loss: 0.0551
Epoch 4/100
Epoch 00004: val_loss improved from 0.05513 to 0.04377, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.0521 - val_loss: 0.0438
Epoch 5/100
Epoch 00005: val_loss improved from 0.04377 to 0.03785, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.0436 - val_loss: 0.0378
Epoch 6/100
Epoch 00006: 

Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.0264 - val_loss: 0.0243
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.0265 - val_loss: 0.0241
Epoch 57/100
Epoch 00057: val_loss improved from 0.02413 to 0.02413, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.0265 - val_loss: 0.0241
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.0265 - val_loss: 0.0243
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.0264 - val_loss: 0.0245
Epoch 60/100
Epoch 00060: val_loss improved from 0.02413 to 0.02412, saving model to ordinal_regression_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 0.0265 - val_loss: 0.0241
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.0264 - val_loss: 0.0242
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.0264 - val_loss: 0.0243
Epoch 63/100
Epoch 00063: val_loss improved from 0.02412 to 0.02405, saving mode

Epoch 18/100
Epoch 00018: val_loss improved from 0.02821 to 0.02811, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.0300 - val_loss: 0.0281
Epoch 19/100
Epoch 00019: val_loss improved from 0.02811 to 0.02795, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.0301 - val_loss: 0.0279
Epoch 20/100
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.0297 - val_loss: 0.0280
Epoch 21/100
Epoch 00021: val_loss improved from 0.02795 to 0.02785, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.0296 - val_loss: 0.0278
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.0296 - val_loss: 0.0283
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.0294 - val_loss: 0.0283
Epoch 24/100
Epoch 00024: val_loss improved from 0.02785 to 0.02775, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.029

Epoch 87/100
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.0283 - val_loss: 0.0269
Epoch 88/100
Epoch 00088: val_loss improved from 0.02687 to 0.02684, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.0282 - val_loss: 0.0268
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.0282 - val_loss: 0.0271
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.0282 - val_loss: 0.0271
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.0282 - val_loss: 0.0270
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.0283 - val_loss: 0.0271
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.0282 - val_loss: 0.0269
Epoch 94/100
Epoch 00094: val_loss improved from 0.02684 to 0.02683, saving model to ordinal_regression_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.0282 - val_loss: 0.0268
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.0282 - val_lo

Epoch 41/100
Epoch 00041: val_loss improved from 0.02347 to 0.02340, saving model to ordinal_regression_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.0253 - val_loss: 0.0234
Epoch 42/100
Epoch 00042: val_loss improved from 0.02340 to 0.02340, saving model to ordinal_regression_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.0252 - val_loss: 0.0234
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.0252 - val_loss: 0.0239
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.0252 - val_loss: 0.0237
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.0253 - val_loss: 0.0246
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.0252 - val_loss: 0.0235
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.0251 - val_loss: 0.0236
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.0252 - val_loss: 0.0236
Epoch 49/100
Epoch 00049: val_loss improved from 0.02340 to 0.02333, saving mode

...

Epoch 00098: val_loss did not improve
 - 2s - loss: 0.0139 - val_loss: 0.0130
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.0138 - val_loss: 0.0131
Epoch 100/100
Epoch 00100: val_loss improved from 0.01287 to 0.01286, saving model to ordinal_regression_flow_ChIJC9G6K_5SUkYRWa6ieQhstlo.best.hdf5
 - 2s - loss: 0.0139 - val_loss: 0.0129
Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.09392, saving model to ordinal_regression_flow_ChIJdbHNKf5SUkYRH2U_kHcp420.best.hdf5
 - 3s - loss: 0.1503 - val_loss: 0.0939
Epoch 2/100
Epoch 00002: val_loss improved from 0.09392 to 0.04731, saving model to ordinal_regression_flow_ChIJdbHNKf5SUkYRH2U_kHcp420.best.hdf5
 - 2s - loss: 0.0732 - val_loss: 0.0473
Epoch 3/100
Epoch 00003: val_loss improved from 0.04731 to 0.03201, saving model to ordinal_regression_flow_ChIJdbHNKf5SUkYRH2U_kHcp420.best.hdf5
 - 2s - loss: 0.0447 - val_loss: 0.0320
Epoch 4/100
Epoch 00004: val_loss improv

Epoch 00055: val_loss did not improve
 - 2s - loss: 0.0147 - val_loss: 0.0129
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.0147 - val_loss: 0.0126
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.0147 - val_loss: 0.0128
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0126
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0129
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0130
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0127
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0131
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.0147 - val_loss: 0.0126
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.0146 - val_loss: 0.0130
Epoch 65/100
Epoch 00065: val_loss improved from 0.01261 to 0.01254, saving model to ordinal_regression

Epoch 20/100
Epoch 00020: val_loss improved from 0.01891 to 0.01885, saving model to ordinal_regression_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 2s - loss: 0.0204 - val_loss: 0.0189
Epoch 21/100
Epoch 00021: val_loss improved from 0.01885 to 0.01885, saving model to ordinal_regression_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 2s - loss: 0.0203 - val_loss: 0.0189
Epoch 22/100
Epoch 00022: val_loss improved from 0.01885 to 0.01883, saving model to ordinal_regression_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 2s - loss: 0.0202 - val_loss: 0.0188
Epoch 23/100
Epoch 00023: val_loss improved from 0.01883 to 0.01869, saving model to ordinal_regression_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 2s - loss: 0.0202 - val_loss: 0.0187
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.0202 - val_loss: 0.0187
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.0201 - val_loss: 0.0187
Epoch 26/100
Epoch 00026: val_loss improved from 0.01869 to 0.01856, s

Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.0192 - val_loss: 0.0180
Epoch 86/100
Epoch 00086: val_loss improved from 0.01789 to 0.01786, saving model to ordinal_regression_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.0193 - val_loss: 0.0179
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0181
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0179
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0180
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0179
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0183
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.0193 - val_loss: 0.0180
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.0193 - val_loss: 0.0179
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.0193 - val_loss: 0

Epoch 42/100
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.0233 - val_loss: 0.0227
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.0232 - val_loss: 0.0225
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.0232 - val_loss: 0.0226
Epoch 45/100
Epoch 00045: val_loss improved from 0.02246 to 0.02236, saving model to ordinal_regression_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 2s - loss: 0.0232 - val_loss: 0.0224
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.0232 - val_loss: 0.0224
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.0231 - val_loss: 0.0224
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.0231 - val_loss: 0.0224
Epoch 49/100
Epoch 00049: val_loss improved from 0.02236 to 0.02230, saving model to ordinal_regression_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 2s - loss: 0.0231 - val_loss: 0.0223
Epoch 50/100
Epoch 00050: val_loss improved from 0.02230 to 0.02227, saving mode

In [42]:
multiple_places(place_groups_junctions, 'place_groups_junctions')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.14295, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 4s - loss: 0.1875 - val_loss: 0.1429
Epoch 2/100
Epoch 00002: val_loss improved from 0.14295 to 0.10168, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.1230 - val_loss: 0.1017
Epoch 3/100
Epoch 00003: val_loss improved from 0.10168 to 0.08037, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.0916 - val_loss: 0.0804
Epoch 4/100
Epoch 00004: val_loss improved from 0.08037 to 0.06894, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.0756 - val_loss: 0.0689
Epoch 5/100
Epoch 00005: val_loss improved from 0.06894 to 0.06176, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.0663 - val_loss: 0.0618
Epoch 6/100
Epoch 00006: 

Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.0437 - val_loss: 0.0425
Epoch 60/100
Epoch 00060: val_loss improved from 0.04250 to 0.04241, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 2s - loss: 0.0437 - val_loss: 0.0424
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.0438 - val_loss: 0.0427
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.0436 - val_loss: 0.0425
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.0437 - val_loss: 0.0425
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.0437 - val_loss: 0.0443
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.0437 - val_loss: 0.0437
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.0436 - val_loss: 0.0426
Epoch 67/100
Epoch 00067: val_loss improved from 0.04241 to 0.04233, saving model to ordinal_regression_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.0437 - val_lo